In [86]:
import time
import numpy as np

###  Utility Function

In [87]:
epsilon = 0.0001
np.random.seed(0)

def _t(x):
    return np.transpose(x)

def _m(A,B):
    return np.matmul(A,B)

def sigmoid(x):
    return 1/ (1+np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))
    

### Define Neuron 

#### numpy function
###### numpy.random.normal(loc=0.0, scale=1.0, size=None)
loc = float or array_like of floats
mean of the distribution

scale = float or array_like of floats
standard deviation(spread or "width") of distribution. Must be non-negative

size = int or tuple of ints, optional
Output shape. if the given shape is `(m,n,k)`, then `m*n*k` smaples are drawn. if size is None, a single value is returned if `loc` and `scale` are both scalars.

###### numpy.zeros_like(a, dtype=None, order='K', subok=True, shape=None)
a = array_like
the shape and data-type of a define these same attributes of the retured array
```python
>>> x = np.arange(6)
>>> x = x.reshape((2, 3))
>>> x
array([[0, 1, 2],
       [3, 4, 5]])

>>> np.zeros_like(x)
array([[0, 0, 0],
       [0, 0, 0]])       
```

In [88]:
class Neuron:
    def __init__(self, W, b, a):
        # Model Parametor
        self.W = W
        self.b = b 
        self.a = a # activation function
        
        # Gradients
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        
    def __call__(self, x):
        return self.a(_m(_t(self.W), x) + self.b) # activation((W^T)x + b)

###  Deep Neural Network

In [89]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i,o)), np.zeros((o,))
        self.sequence = list()
        
        # Fisrt hidden layer
        W, b= init_var(num_input, num_neuron)
        self.sequence.append(Neuron(W,b, activation))
        
        # Hidden layers
        for _ in range(hidden_depth):
            W, b= init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W,b, activation))
        
        # Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Neuron(W,b, activation))
        
    def __call__(self,x):
        for layer in self.sequence:
            x = layer(x)
        return x
    
    def calc_gradient(self, x, y, loss_func):
        def get_new_sequence(layer_index, new_neuron):
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index :
                    new_sequence.append(new_neuron)
                else:
                    new_sequence.append(layer)
            return new_sequence
        
        def eval_sequence(x, sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        # 기준이 되는 현재 상태의 loss 계산
        loss = loss_func(self(x), y) # 추정값, 결과값
        # loss = loss_func(self.__init__(x), y) 과 동일
        
        for layer_id, layer in enumerate(self.sequence):
            for w_i, w in enumerate(layer.W): # iterate W (row)
                for w_j, ww in enumerate(w): # iterate w (col)
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon
                    
                    new_neuron = Neuron(W, layer.b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    num_grad = (loss_func(h,y) - loss) / epsilon # f(x+eps) - f(x) / eps
                    layer.dW[w_i][w_j] = num_grad
                for b_i, bb in enumerate(layer.b): # iterate w (col)
                    b = np.copy(layer.b)
                    b[b_i] = bb + epsilon
                    
                    new_neuron = Neuron(layer.W, b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    num_grad = (loss_func(h,y) - loss) / epsilon # f(x+eps) - f(x) / eps
                    layer.db[b_i] = num_grad
        return loss

###  Gradient Descent

In [90]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x,y,loss_obj)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

###  동작 테스트

In [91]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.2601871301682722
Epoch 1: Test loss 0.25872109434802293
Epoch 2: Test loss 0.2572633168616838
Epoch 3: Test loss 0.2558138966963392
Epoch 4: Test loss 0.25437292852721827
Epoch 5: Test loss 0.2529405027235575
Epoch 6: Test loss 0.25151670535955306
Epoch 7: Test loss 0.2501016182301732
Epoch 8: Test loss 0.24869531887173726
Epoch 9: Test loss 0.24729788058695665
Epoch 10: Test loss 0.24590937247451616
Epoch 11: Test loss 0.24452985946274358
Epoch 12: Test loss 0.2431594023473848
Epoch 13: Test loss 0.24179805783320063
Epoch 14: Test loss 0.240445878579229
Epoch 15: Test loss 0.23910291324750862
Epoch 16: Test loss 0.23776920655513684
Epoch 17: Test loss 0.2364447993293322
Epoch 18: Test loss 0.2351297285654843
Epoch 19: Test loss 0.2338240274878536
Epoch 20: Test loss 0.23252772561280946
Epoch 21: Test loss 0.23124084881437787
Epoch 22: Test loss 0.22996341939197767
Epoch 23: Test loss 0.22869545614004272
Epoch 24: Test loss 0.22743697441951077
Epoch 25: Test loss 0

In [41]:
np.random.normal(0.0,1.0,(10,))

array([-0.78310075,  0.48886626, -0.93393626, -0.71124334, -1.01567754,
       -0.24991041, -0.46448255, -1.45380872, -0.19186241, -1.24207341])

In [45]:
np.random.normal(0.0,1.0,(10,))

array([-1.61164024,  0.39903648, -0.23499778, -0.48118335,  1.13834965,
        1.28179925, -0.59266593, -1.6601261 ,  1.31482232, -1.52167554])